# Class

In [ ]:
# Please run the class first and then app 

In [ ]:
# pip statements to install dependencies in case its missing

# for SentenceTransformer
pip install sentence-transformers

# for dash and its components
pip install dash dash-core-components dash-html-components dash-table jupyter-dash

# for pandas, sklearn, scipy, fuzzywuzzy and nltk
pip install pandas scikit-learn scipy fuzzywuzzy nltk numpy plotly

pip install python-Levenshtein

In [4]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cdist
from fuzzywuzzy import fuzz
import nltk
import numpy as np
import plotly.graph_objs as go
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np

class StringMatching:
    def __init__(self, source, target):
        # default column pairings
        self.default_source_cols = ['Variable', 'Label', 'Definition (derived Vars)']
        self.default_target_cols = ['Variable', 'Label', 'Rule for derivation']

        source_cols = [col for col in self.default_source_cols if col in source.columns]
        target_cols = [col for col in self.default_target_cols if col in target.columns]

        self.source = source.dropna(subset=source_cols)
        self.target = target.dropna(subset=target_cols)
        
        self.model = SentenceTransformer('bert-base-uncased')

        # Calculate the embeddings for cleaned data
        self.source_embeddings = self.get_embeddings(self.clean_data(self.source[source_cols]))
        self.target_embeddings = self.get_embeddings(self.clean_data(self.target[target_cols]))

        # For tfidf similarity
        self.corpus = pd.concat([self.source[source_cols], self.target[target_cols]])
        self.corpus = self.corpus.replace(np.nan, '').dropna()
        self.tfidf_vectorizer = TfidfVectorizer().fit(self.corpus)


    @staticmethod
    def clean_data(data):
        stop_words = ['a', 'an', 'the', 'in', 'on', 'of', 'and', 'or', 'is', 'are', 'to', 'for', 'it', 'that', 'this']
        stop_words += ['<', '%', ':', '&']  # Add any additional stopwords specific to your data
        return [' '.join([word for word in str(row).lower().split() if word not in stop_words]) for row in data]

    def get_embeddings(self, data):
        return self.model.encode(data)  
    #below are the NLP methods
    @staticmethod
    def similarity_token_sort(str1, str2):
        score = fuzz.token_sort_ratio(str1, str2)
        return score

    @staticmethod
    def similarity_ratio(str1, str2):
        score = fuzz.ratio(str1, str2)
        return score

    @staticmethod
    def similarity_partial_ratio(str1, str2):
        score = fuzz.partial_ratio(str1, str2)
        return score

    @staticmethod
    def similarity_token_sort_ratio(str1, str2):
        score = fuzz.token_sort_ratio(str1, str2)
        return score

    @staticmethod
    def similarity_token_set_ratio(str1, str2):
        score = fuzz.token_set_ratio(str1, str2)
        return score

    @staticmethod
    def similarity_partial_token_set_ratio(str1, str2):
        score = fuzz.partial_token_set_ratio(str1, str2)
        return score
    @staticmethod
    def similarity_partial_token_sort_ratio(str1, str2):
        score = fuzz.partial_token_sort_ratio(str1, str2)
        return score
    @staticmethod
    def QRatio(str1, str2):
        score = fuzz.QRatio(str1, str2)
        return score

    @staticmethod
    def UQRatio(str1, str2):
        score = fuzz.UQRatio(str1, str2)
        return score

    @staticmethod
    def UWRatio(str1, str2):
        score = fuzz.UWRatio(str1, str2)
        return score

    @staticmethod
    def WRatio(str1, str2):
        score = fuzz.WRatio(str1, str2)
        return score
    
    def tfidf_similarity(self, text1, text2):
        tfidf1 = self.tfidf_vectorizer.transform([text1])
        tfidf2 = self.tfidf_vectorizer.transform([text2])
        return cosine_similarity(tfidf1, tfidf2)[0][0]
    
    def bert_similarity(self, text1, text2):
        embeddings1 = self.get_embeddings(self.clean_data([text1]))
        embeddings2 = self.get_embeddings(self.clean_data([text2]))
        # We use cosine distance, so we subtract from 1 to get similarity
        similarity = 1 - cdist(embeddings1, embeddings2, 'cosine')[0][0]
        return similarity

 #method that does the calcualtions and filetring based on user input 
    def multi_score(self, n, similarity_func, source_cols=None, target_cols=None, source_filter=None, 
                source_file_name="Source", target_file_name="Target"):
        matches = []
        source_df = self.source
        target_df = self.target
        # Strip white spaces from object type columns
        source_df = source_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

        # Use default columns if none provided
        if source_cols is None:
            source_cols = ['Variable', 'Label', 'Definition (derived Vars)']

        if target_cols is None:
            target_cols = ['Variable', 'Label', 'Rule for derivation']

        # If a filter is provided, apply it to the source dataframe
        if source_filter is not None:
            col, val = source_filter
            source_df = source_df[source_df[col] == val]

        for _, row1 in source_df.iterrows():
            data1 = [row1[col] for col in source_cols]
            scores = []

            for _, row2 in target_df.iterrows():
                data2 = [row2[col] for col in target_cols]
                scores.append(tuple(similarity_func(d1, d2) for d1, d2 in zip(data1, data2)))

            scores_sorted = sorted(enumerate(scores), key=lambda x: sum(x[1]), reverse=True)

            if scores_sorted:
                top_matches = [tuple(data1) + tuple(target_df.iloc[idx][target_cols]) + tuple(score) 
                               for idx, score in scores_sorted[:n]]
                matches.extend(top_matches)

        columns = [f'{source_file_name} {col}' for col in source_cols] + \
                   [f'{target_file_name} {col}' for col in target_cols] + \
                   [f'{source_file_name} {s_col} - {target_file_name} {t_col} Score' 
                    for s_col, t_col in zip(source_cols, target_cols)]
        result_df = pd.DataFrame(matches, columns=columns)

        # Filter the columns that end with "Score" and plot them
        score_columns = [col for col in result_df.columns if col.endswith('Score')]
        scores_plot = go.Figure()
        for col in score_columns:
            for source_col, target_col in zip(source_cols, target_cols):
                scores_plot.add_trace(go.Scatter(
                    x=result_df[col].tolist(),
                    y=result_df[f'{source_file_name} {source_col}'].tolist(),
                    name=f'{source_col} - {target_col}',
                    hoverinfo='text',
                    text=[f'{source_file_name}: {source_value}<br>{target_file_name}: {target_value}' 
                          for source_value, target_value in zip(
                              result_df[f'{source_file_name} {source_col}'].tolist(), 
                              result_df[f'{target_file_name} {target_col}'].tolist()
                          )]
                ))

        scores_plot.update_layout(
            title="Scores for Similarity Functions",
            xaxis_title=f"{source_file_name} - {target_file_name} Scores",
            yaxis_title="Score",
            hovermode="closest",
            barmode="group"
        )

        return result_df, scores_plot
    
   
    #calucalte similiarty scores based on what function is selected
    def similarity_scores(self, n):
        similarity_functions = [
            ('similarity_ratio', self.similarity_ratio),
            ('similarity_partial_ratio', self.similarity_partial_ratio),
            ('similarity_token_sort_ratio', self.similarity_token_sort_ratio),
            ('similarity_token_set_ratio', self.similarity_token_set_ratio),
            ('similarity_partial_token_set_ratio', self.similarity_partial_token_set_ratio),
            ('similarity_partial_token_sort_ratio', self.similarity_partial_token_sort_ratio),
            ('QRatio', self.QRatio),
            ('UQRatio', self.UQRatio),
            ('UWRatio', self.UWRatio),
            ('WRatio', self.WRatio),
            ('tfidf_similarity', self.tfidf_similarity) 
            #,('bert_similarity', self.bert_similarity)
        ]

        # Dataframe to hold the results
        scores_df = pd.DataFrame()

        for _, row1 in self.df1.iterrows():
            var1, label1 = row1['Variable'], row1['Label']
            matched_vars = []
            matched_labels = []

            for _, row2 in self.df2.iterrows():
                var2, label2 = row2['Variable'], row2['Label']
                scores = {}

                for func_name, similarity_func in similarity_functions:
                    label_score = similarity_func(label1, label2)
                    scores[func_name] = label_score

                # Add the variable and matched variable to the scores dict
                scores['EU_Variable'] = var1
                scores['Japan_Variable'] = var2
                scores['EU_Label'] = label1
                scores['Japan_Label'] = label2
                matched_vars.append(scores)

            # Add the scores for this EU_Variable to the dataframe
            scores_df = scores_df.append(pd.DataFrame(matched_vars), ignore_index=True)

        # Set the order of the columns
        columns_order = ['EU_Variable', 'EU_Label', 'Japan_Variable', 'Japan_Label'] + [func_name for func_name, _ in similarity_functions]
        scores_df = scores_df[columns_order]

        return scores_df

In [ ]:
Eu = pd.read_csv('EU_all_variables.csv')

Jp= pd.read_csv('Japan_all_variables.csv')

In [ ]:
Jp.columns

In [ ]:
matcher= StringMatching(source=Jp,target=Eu)

In [ ]:
result_df, scores_plot = matcher.multi_score(
    n=5,
    similarity_func=matcher.similarity_partial_ratio,
    source_cols=['Variable', 'Label'],
    target_cols=['Variable','Label'],
    source_filter=('Variable', 'STUDYID')
)

In [ ]:
print("Result DataFrame:")
display(result_df)
scores_plot.show()

# App
# Please run the Class above first.

In [5]:
import logging
import dash_bootstrap_components as dbc
import base64
import datetime
import io
import dash
from jupyter_dash import JupyterDash
from dash import no_update
from dash.dependencies import Input, Output, State
from dash import dash_table
from dash import dcc
import plotly.express as px
from dash import html
import pandas as pd

logging.basicConfig(filename='app.log', filemode='w', format='%(name)s - %(levelname)s - %(message)s', level=logging.INFO)
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets,
                suppress_callback_exceptions=True)

app.layout = html.Div([  # App layout opening

    dcc.Tabs([  # Tabs opening

        dcc.Tab(label='Upload', children=[
            dcc.Loading(  # Loading opening
                id="loading",
                type="circle", 
                children=[  # Loading children opening
                    dcc.Upload(  # Upload opening
                        id='upload-data',
                        children=html.Div([
                            'Drag and Drop or ',
                            html.A('Select Files')
                        ]),
                        style={
                            'width': '100%',
                            'height': '60px',
                            'lineHeight': '60px',
                            'borderWidth': '1px',
                            'borderStyle': 'dashed',
                            'borderRadius': '5px',
                            'textAlign': 'center',
                            'margin': '10px'
                        },
                        multiple=True
                    ),  # Upload closing
                    html.Div(id='output-div'),
                ]  # Loading children closing
            ),  # Loading closing
        ]),

        dcc.Tab(label='Data', children=[

            html.Div(id='output-datatable'),
        ]),

        dcc.Tab(label='Matching', children=[
            html.Div([
                html.Label('Select similarity function:'),
                dcc.Dropdown(
                    id='similarity-function-dropdown',
                    options=[
                        {'label': 'similarity_ratio', 'value': 'similarity_ratio'},
                        {'label': 'similarity_partial_ratio', 'value': 'similarity_partial_ratio'},
                        {'label': 'similarity_token_sort_ratio', 'value': 'similarity_token_sort_ratio'},
                        {'label': 'similarity_token_set_ratio', 'value': 'similarity_token_set_ratio'},
                        {'label': 'similarity_partial_token_set_ratio', 'value': 'similarity_partial_token_set_ratio'},
                        {'label': 'similarity_partial_token_sort_ratio', 'value': 'similarity_partial_token_sort_ratio'},
                        {'label': 'QRatio', 'value': 'QRatio'},
                        {'label': 'UQRatio', 'value': 'UQRatio'},
                        {'label': 'UWRatio', 'value': 'UWRatio'},
                        {'label': 'WRatio', 'value': 'WRatio'},
                        {'label': 'tfidf_similarity', 'value': 'tfidf_similarity'},
                        {'label': 'bert_similarity', 'value': 'bert_similarity'}],  # Uncomment if you want to include
                    value='similarity_ratio'
                ),
            ]),
             # Dropdown to select source file
            html.Div([
                html.Label('Select Source File:'),
                dcc.Dropdown(
                    id='source-file-dropdown',
                    options=[],
                    placeholder='Select Source File',
                ),
            ]),
            # Dropdown to select target file
            html.Div([
                html.Label('Select Target File:'),
                dcc.Dropdown(
                    id='target-file-dropdown',
                    options=[],
                    placeholder='Select Target File',
                ),
            ]),
            html.Div([
                html.Label('Enter Source column name:'),
                dcc.Input(
                    id='source-column',
                    type='text',
                    placeholder='Enter source column name',
                ),html.P("Example: Variable, Label (case matters)", style={'color': 'grey'}),
            ]),
            html.Div([
                html.Label('Enter Target column name:'),
                dcc.Input(
                    id='target-column',
                    type='text',
                    placeholder='Enter target column name',
                ),html.P("Example: Variable, Label (case matters)", style={'color': 'grey'}),
            ]),
            html.Div([
                html.Label('Enter value to match:'),
                dcc.Input(
                    id='input-value',
                    type='text',
                    placeholder='Enter value to match',
                ),html.P("Example: Column name,Variable in column - Variable, USDYID", style={'color': 'grey'}),
            ]),
            html.Button('Run multi score', id='run-button', n_clicks=0),
            html.Div(id='matching-output'),
            html.H3("Scores Plot"),
            dcc.Graph(id='graph-output')
        ])  # End of 'Matching' Tab
    ]),  # End of Tabs

    html.Div(id='store-container-1', style={'display': 'none'}),
    html.Div(id='store-container-2', style={'display': 'none'}),
    dcc.Store(id='store-df1'),
    dcc.Store(id='store-df2')

]) # end of layout
#dictionary of the available methods if you decide to add more please update this 
similarity_functions = {
    'similarity_ratio': StringMatching.similarity_ratio,
    'similarity_partial_ratio': StringMatching.similarity_partial_ratio,
    'similarity_token_sort_ratio': StringMatching.similarity_token_sort_ratio,
    'similarity_token_set_ratio': StringMatching.similarity_token_set_ratio,
    'similarity_partial_token_set_ratio': StringMatching.similarity_partial_token_set_ratio,
    'similarity_partial_token_sort_ratio': StringMatching.similarity_partial_token_sort_ratio,
    'QRatio': StringMatching.QRatio,
    'UQRatio': StringMatching.UQRatio,
    'UWRatio': StringMatching.UWRatio,
    'WRatio': StringMatching.WRatio,
    'tfidf_similarity': StringMatching.tfidf_similarity,
    'bert_similarity': StringMatching.bert_similarity  # Uncomment if you want to include
}

# the callback and the method that updates the ouput in the- data tab
@app.callback([Output('store-df1', 'data'), Output('store-df2', 'data'), Output('output-datatable', 'children')],
              Input('upload-data', 'contents'),
              State('upload-data', 'filename'),
              State('upload-data', 'last_modified'))
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None and len(list_of_contents) >= 2:
        df1 = parse_contents(list_of_contents[0], list_of_names[0], list_of_dates[0])
        df2 = parse_contents(list_of_contents[1], list_of_names[1], list_of_dates[1])
        
        if df1 is not None and df2 is not None:
            children = html.Div([
                html.H3(f"{list_of_names[0]}"),
                dash_table.DataTable(
                    id='table1',
                    columns=[{"name": i, "id": i} for i in df1.columns],
                    data=df1.to_dict('records'),
                    filter_action='native', 
                    style_table={'overflowX': 'auto'},
                    style_cell={
                        'minWidth': '100px', 'width': '100px', 'maxWidth': '100px',
                        'overflow': 'hidden',
                        'textOverflow': 'ellipsis',
                    },
                    page_size=10, 
                    style_header={
                        'backgroundColor': 'rgb(230, 230, 230)',
                        'fontWeight': 'bold'
                    },
                    style_data_conditional=[
                        {
                            'if': {'state': 'active'}, 
                            'backgroundColor': 'rgba(0, 116, 217, 0.3)',
                            'border': '1px solid rgb(0, 116, 217)'
                        }
                    ]
                ),
                html.H3(f"{list_of_names[1]}"),
                dash_table.DataTable(
                    id='table2',
                    columns=[{"name": i, "id": i} for i in df2.columns],
                    data=df2.to_dict('records'),
                    filter_action='native', 
                    style_table={'overflowX': 'auto'},
                    style_cell={
                        'minWidth': '100px', 'width': '100px', 'maxWidth': '100px',
                        'overflow': 'hidden',
                        'textOverflow': 'ellipsis',
                    },
                    page_size=10, 
                    style_header={
                        'backgroundColor': 'rgb(230, 230, 230)',
                        'fontWeight': 'bold'
                    },
                    style_data_conditional=[
                        {
                            'if': {'state': 'active'}, 
                            'backgroundColor': 'rgba(0, 116, 217, 0.3)',
                            'border': '1px solid rgb(0, 116, 217)'
                        }
                    ]
                )
            ])
            return df1.to_dict('records'), df2.to_dict('records'), children

    return dash.no_update, dash.no_update, dash.no_update

#callback and method to run the muli_score function from the class this is what is triggered when the button is clicked - matching tab
@app.callback(
    [Output('matching-output', 'children'), Output('graph-output', 'figure')],
    [Input('run-button', 'n_clicks')],
    [State('similarity-function-dropdown', 'value'),
     State('source-column', 'value'),
     State('target-column', 'value'),
     State('input-value', 'value'),
     State('source-file-dropdown', 'value'),  # Add this State for source file dropdown
     State('target-file-dropdown', 'value'),  # Add this State for target file dropdown
     State('store-df1', 'data'),
     State('store-df2', 'data'),
     State('upload-data', 'filename')]  # Add this State for filenames
)
def run_multi_score(n_clicks, similarity_function, source_column, target_column, input_value,
                    source_file, target_file, df1_data, df2_data, list_of_names):  
    if n_clicks > 0:
        logging.info('run_multi_score called with n_clicks=%s, similarity_function=%s', n_clicks, similarity_function)
        try:
            # convert input string to list of strings
            source_cols = [col.strip() for col in source_column.split(',')]
            target_cols = [col.strip() for col in target_column.split(',')]
            # Determine which dataframe corresponds to the selected source and target files
            if source_file == list_of_names[0].split('.')[0]:
                source_df = pd.DataFrame(df1_data)
                target_df= pd.DataFrame(df2_data)
            else:
                source_df = pd.DataFrame(df2_data)
                target_df = pd.DataFrame(df1_data)
            print("Source DataFrame columns:", source_df.columns)
            print("Target DataFrame columns:", target_df.columns)
            matcher = StringMatching(source_df, target_df)

            # Parse the input value to separate column name and variable
            input_column, input_variable = [val.strip() for val in input_value.split(',')]

            # convert input_variable to appropriate type
            dtype = source_df[input_column].dtype
            if np.issubdtype(dtype, np.number):
                input_variable = float(input_variable)
            elif dtype == 'bool':
                input_variable = bool(input_variable)

            result_df, scores_plot = matcher.multi_score(
                n=5,  
                similarity_func=similarity_functions[similarity_function],
                source_cols=source_cols,  
                target_cols=target_cols,  
                source_filter=(input_column, input_variable)
            )# output format for the result
            result_table = dash_table.DataTable(
                    id='table',
                    columns=[{"name": i, "id": i} for i in result_df.columns],
                    data=result_df.to_dict('records'),
                    filter_action='native', 
                    style_table={'overflowX': 'auto'},
                    style_cell={
                        'minWidth': '100px', 'width': '100px', 'maxWidth': '100px',
                        'overflow': 'hidden',
                        'textOverflow': 'ellipsis',
                    },
                    page_size=10, 
                    style_header={
                        'backgroundColor': 'rgb(230, 230, 230)',
                        'fontWeight': 'bold'
                    },
                    style_data_conditional=[
                        {
                            'if': {'state': 'active'}, 
                            'backgroundColor': 'rgba(0, 116, 217, 0.3)',
                            'border': '1px solid rgb(0, 116, 217)'
                        }
                    ]
                )

            return result_table, scores_plot
        except Exception as e:
            error_message = f"Error: {str(e)}"
            logging.error(error_message)
            return html.Div([error_message]), dash.no_update
    return dash.no_update, dash.no_update



#parse content based on file upload
def parse_contents(contents, filename, date):
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    try:
        if 'csv' in filename:
            df = pd.read_csv(io.StringIO(decoded.decode('utf-8')),header=0)
        elif 'xls' in filename:
            df = pd.read_excel(io.BytesIO(decoded))
        else:
            return None
    except Exception as e:
        print(e)
        return None

    return df
#this is the callback and hte method to update the drop down that helps get the names of the files- matching tab
@app.callback(
    Output('source-file-dropdown', 'options'),
    Output('target-file-dropdown', 'options'),
    [Input('upload-data', 'contents')],
    [State('upload-data', 'filename')]
)
def update_file_dropdowns(list_of_contents, list_of_names):
    if list_of_contents is not None and len(list_of_contents) >= 2:
        file_options = [{'label': name.split('.')[0], 'value': name.split('.')[0]} for name in list_of_names]
        return file_options, file_options
    return [], []  # Empty options if no files uploaded



# change the port number if the error says port already occupied
if __name__ == '__main__':
    app.run_server(debug=False ,port=8050)



D:\conda\lib\site-packages\dash\dash.py:516: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



Dash app running on http://127.0.0.1:8050/
Source DataFrame columns: Index(['Domain', 'Variable', 'Label', 'Type', 'Codes', 'Rule for derivation'], dtype='object')
Target DataFrame columns: Index(['ADS Name', 'Variable', 'Label', 'Definition (derived Vars)', 'Type'], dtype='object')


Some weights of the model checkpoint at C:\Users\suraj/.cache\torch\sentence_transformers\bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]